In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QComboBox, QPushButton, QVBoxLayout, QWidget, QFileDialog, QMessageBox
from PyQt5.QtGui import QPixmap
import cv2
import numpy as np
#from fgp_algorithm import fgp_denoise
from admm_algorithm import ADMM_gray
#from gd_algorithm import gd_denoise
#from pdn_algorithm import pdn_denoise
#from asd_algorithm import asd_denoise
#from fgh_algorithm import fgh_denoise


class ImageDenoisingApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Image Denoising App")
        self.image_path = None
        self.algorithm = None

        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        self.image_label = QLabel(self)
        self.layout.addWidget(self.image_label)

        self.image_type_combo = QComboBox(self)
        self.image_type_combo.addItem("Black and White")
        self.image_type_combo.addItem("Colour")
        self.image_type_combo.currentIndexChanged.connect(self.image_type_changed)
        self.layout.addWidget(self.image_type_combo)

        self.algorithm_combo = QComboBox(self)
        self.layout.addWidget(self.algorithm_combo)

        self.upload_button = QPushButton("Upload Image", self)
        self.upload_button.clicked.connect(self.upload_image)
        self.layout.addWidget(self.upload_button)

        self.process_button = QPushButton("Process Image", self)
        self.process_button.clicked.connect(self.process_image)
        self.layout.addWidget(self.process_button)

        self.download_button = QPushButton("Download Image", self)
        self.download_button.clicked.connect(self.download_image)
        self.layout.addWidget(self.download_button)

        self.update_algorithm_options("Black and White")

    def image_type_changed(self, index):
        image_type = self.image_type_combo.itemText(index)
        self.update_algorithm_options(image_type)

    def update_algorithm_options(self, image_type):
        self.algorithm_combo.clear()
        if image_type == "Black and White":
            self.algorithm_combo.addItem("FGP")
            self.algorithm_combo.addItem("ADMM")
            self.algorithm_combo.addItem("GD")
            self.algorithm_combo.addItem("PDN")
        else:
            self.algorithm_combo.addItem("ASD")
            self.algorithm_combo.addItem("FGH")

    def upload_image(self):
        file_dialog = QFileDialog(self)
        self.image_path, _ = file_dialog.getOpenFileName(self, "Upload Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if self.image_path:
            pixmap = QPixmap(self.image_path)
            self.image_label.setPixmap(pixmap.scaled(400, 400))

    def process_image(self):
        if not self.image_path:
            QMessageBox.warning(self, "Warning", "Please upload an image first.")
            return

        

        if self.image_type_combo.currentText() == "Black and White":
            image = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
            self.algorithm = self.algorithm_combo.currentText()
            if self.algorithm == "FGP":
                denoised_image = fgp_denoise(image)
            elif self.algorithm == "ADMM":
                denoised_image = ADMM_gray(image, 10, 100)
            elif self.algorithm == "GD":
                denoised_image = gd_denoise(image)
            elif self.algorithm == "PDN":
                denoised_image = pdn_denoise(image)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return
        else:
            self.algorithm = self.algorithm_combo.currentText()
            image = cv2.imread(self.image_path)
            if self.algorithm == "ASD":
                denoised_image = asd_denoise(image)
            elif self.algorithm == "FGH":
                denoised_image = fgh_denoise(image)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return
        denoised_image = denoised_image.astype(np.uint8)
        cv2.imwrite("denoised_image.jpg", denoised_image)
        pixmap = QPixmap("denoised_image.jpg")
        self.image_label.setPixmap(pixmap.scaled(400, 400))

    def download_image(self):
        if not self.image_path:
            QMessageBox.warning(self, "Warning", "No denoised image available.")
            return

        file_dialog = QFileDialog(self)
        file_dialog.setAcceptMode(QFileDialog.AcceptSave)
        file_path, _ = file_dialog.getSaveFileName(self, "Save Denoised Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if file_path:
            cv2.imwrite(file_path, cv2.imread("denoised_image.jpg"))
            QMessageBox.information(self, "Success", "Denoised image saved successfully.")

    def show(self):
        super().show()
        self.resize(600, 600)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    image_denoising_app = ImageDenoisingApp()
    image_denoising_app.show()
    sys.exit(app.exec_())


/Users/liangxinyu/Desktop/image_denoising/admm_algorithm.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[:,:,0] = x - np.roll(x, (-1, 0), axis=(0, 1))
/Users/liangxinyu/Desktop/image_denoising/admm_algorithm.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  grad[:, :, 1] = x - np.roll(x, (0, -1), axis=(0, 1))
/var/folders/z0/6n18ndds33nfnl2t3_9xbyy80000gn/T/ipykernel_42094/1830492493.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
  denoised_image = denoised_image.astype(np.uint8)
